# Урок 2. Работа с данными.

In [ ]:
import Pkg;
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("DelimitedFiles")

In [ ]:
using CSV  # Чтение и запись CSV файлов
using DataFrames  # Create a data structure
using DelimitedFiles # Работа с файлами с разделителями
using Statistics # Модуль для работы со статистическими данными
# using XLSX

## 1. Линейная алгебра

In [ ]:
A = rand(5,5); # Создадим матрицу размером 5x5
A_t = A' # Транспонирование матрицы
A = A_t*A; # Умножение матриц
A

- `size()` Покажет размерность матрицы
- `sizeof()` Покажет размер в байтах, а `typeof()` тип данных

In [ ]:
size(A), sizeof(A), typeof(A)

Обращаться к матрице можно по индексу строки и столбца, например так (Обратите внимание, они считаются с 1):

In [ ]:
A[2,2]

Или представить, что у нас длинный вектор (аналог операции `flatten`) и задать одним индексом:

In [ ]:
A[7]

## 2. Чтение файлов

Возьмем [отсюда](https://www.kaggle.com/shivamb/netflix-shows) датасет и скачаем его, например так:

In [ ]:
download("https://raw.githubusercontent.com/JuliaEvangelists/Julia-in-DS/main/data/netflix_titles.csv",
    "netflix.csv")

Для чтения файлов с разделителями можно использовать стандартную библиотеку [DelimitedFiles](https://docs.julialang.org/en/v1/stdlib/DelimitedFiles/) и метода `readdlm`:

In [ ]:
netflix, header = DelimitedFiles.readdlm("netflix.csv", ',', header=true)

println("header это: ", typeof(header), " и имеет размерность ", size(header))
println("netflix это: ", typeof(netflix), " и имеет размерность ", size(netflix))

В Python мы привыкли работать с табличными данными в виде Dataframe'ов. По аналогии с Pandas, пакет [DataFrames](https://juliadata.github.io/DataFrames.jl/stable/man/getting_started/) позволяет рабоотать с табличными данными придставляя их в виде Dataframe'ов. Объекты типа DataFrame представляют таблицу данных как серию векторов, каждый из которых соответствует столбцу или переменной. 

Создать DataFrame, можно просто передав кажду колонку как вектор: 

In [ ]:
df = DataFrame(Month = ["Jan", "Feb", "March"], Value = 1:3)

Посмотрим на тип объекта `df`:

In [ ]:
typeof(df)

Теперь можно добавлять столбцы или строки:

In [ ]:
old_size = size(df)
df.Year = ones(3)*2020
col_add_size = size(df)
print("фрейм размером ",old_size, " стал ",  col_add_size)
df

In [ ]:
push!(df, ("April", 4, 2020))
print("фрейм размером ",col_add_size, " стал ",  size(df))

Конечно мы можем создать DataFrame и на основе матрицы:

In [ ]:
netflix_df = DataFrame(netflix)
first(netflix_df, 3)

In [ ]:
last(netflix_df, 3)

Лучшим вариантом создать DataFrame может быть прямо при чтении CSV файла c помощью пакета [CSV](https://juliadata.github.io/CSV.jl/stable/)     

In [ ]:
netflix_csv = CSV.read("netflix.csv");
typeof(netflix_csv), typeof(netflix_df)

In [ ]:
first(netflix_csv, 3)

## 3. Срезы и обращение к данным

In [ ]:
names(netflix_csv) # Column names can be obtained as strings using the names function:

In [ ]:
propertynames(netflix_csv) # To get column names as Symbols use the propertynames function:

In [ ]:
netflix_csv[1:3, :]

In [ ]:
netflix_csv[[1, 5, 10], :]

In [ ]:
netflix_csv[[1,2,3,4,5], [:country, :director, :rating, :duration]]

In [ ]:
netflix_csv[netflix_csv.release_year .=2000, [:director, :rating]]

In [ ]:
netflix_csv[netflix_csv.country .="United States", [:director, :rating]]

In [ ]:
descr = describe(netflix_csv)
names(descr)

In [ ]:
descr[[:variable, :eltype]]

## 4. Аггрегация данных

In [ ]:
colors = DataFrame(color_id = 1:5, color=["Черный", "Голубой", "Белый", "Желтый", "Амарантово-Пурпурный"])

socks = DataFrame(socks_id = 1:8,
    name = ["Снежана", "Кирилл", "Иосиф", "Инокентий","Африкан","Аркадий", "Маруся", "Вальдемар"],
    color_id = [1,2,1,3,1,5,4,2])

qt_washing = DataFrame(socks_id = [1,2,3,4,5,6,7,8], qt_wash = [10,20,15,42,12,17,27,32])
print()

In [ ]:
socks_with_color = innerjoin(socks, colors, on="color_id")

In [ ]:
full_info = leftjoin(socks_with_color[:, [:socks_id, :name, :color]], qt_washing, on="socks_id")

In [ ]:
grouped = groupby(full_info, :color)

In [ ]:
combine(grouped, nrow)

In [ ]:
combine(grouped, nrow, :qt_wash => mean => :mean)

Для более сложных манипуляций с DataFrame'ами существуют разные модули, например: [DataFramesMeta.jl](https://github.com/JuliaData/DataFramesMeta.jl) and [Query.jl](http://www.queryverse.org/Query.jl/stable/). 